# Monitoramento com TensorBoard — CNN em FashionMNIST

Notebook didático em Português. Objetivo: demonstrar logs em TensorBoard (loss, accuracy) durante treino de uma CNN moderada no dataset FashionMNIST.

In [ ]:
%pip install -q torch torchvision tensorboard


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1) Imports e configuração

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
from torch.utils.tensorboard import SummaryWriter

device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)

Device: cpu


## 2) Dataset — FashionMNIST

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

print('Train size:', len(trainset), 'Test size:', len(testset))

Train size: 60000 Test size: 10000


## 3) Modelo CNN moderado

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 10)
        )
    def forward(self, x): return self.net(x)

model = SimpleCNN().to(device)
print(model)

SimpleCNN(
  (net): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=128, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.3, inplace=False)
    (10): Linear(in_features=128, out_features=10, bias=True)
  )
)


## 4) Treinamento com TensorBoard logging

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
writer = SummaryWriter('runs/fashion_cnn_demo')

def evaluate(loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            _, preds = torch.max(out, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return correct / total

epochs = 10
for epoch in range(1, epochs+1):
    model.train()
    running_loss = 0.0
    total, correct = 0, 0
    for i, (imgs, labels) in enumerate(trainloader, 1):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(out, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
        if i % 100 == 0:
            print(f'Batch {i} loss {running_loss/i:.4f}')
    train_acc = correct/total
    val_acc = evaluate(testloader)
    avg_loss = running_loss / len(trainloader)
    print(f'Epoch {epoch}: loss={avg_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')
    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/val', val_acc, epoch)

    # Tente outras formas de visualização
    # writer.add_scalar('Loss/train', avg_loss, epoch)    

writer.close()

Batch 100 loss 0.1454
Batch 200 loss 0.1408
Batch 300 loss 0.1397
Batch 400 loss 0.1419
Epoch 1: loss=0.1428, train_acc=0.9475, val_acc=0.9234
Batch 100 loss 0.1261
Batch 200 loss 0.1277
Batch 300 loss 0.1281
Batch 400 loss 0.1284


KeyboardInterrupt: 

## 5) Visualizar no TensorBoard
No notebook você pode usar a mágica do tensorboard (se o ambiente suportar):
```
%load_ext tensorboard
%tensorboard --logdir runs
```
Ou no terminal:
```
tensorboard --logdir runs
```
Abra o link fornecido para ver Loss/Accuracy ao longo das épocas.

## 6) Experimentos sugeridos
- Alterar `lr` e comparar curvas
- Trocar otimizador (SGD + momentum)
- Salvar checkpoints com `torch.save` e plotar comparação entre runs
- Usar augmentations em `transform` e observar efeito na generalização